In [1]:
import pandas as pd
import textwrap


In [2]:
homicide_df = pd.read_csv('homicide-data2.csv')
homicide_df.tail()

,uid,reported_date,victim_last,victim_first,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition
52174,Was-001380,20160908,WILLIAMS,EVAN,Black,29,Male,Washington,DC,38.828704,-77.002075,Closed by arrest
52175,Was-001381,20160913,SMITH,DEON,Black,19,Male,Washington,DC,38.822852,-77.001725,Open/No arrest
52176,Was-001382,20161114,WASHINGTON,WILLIE,Black,23,Male,Washington,DC,38.828025,-77.002511,Open/No arrest
52177,Was-001383,20161130,BARNES,MARCUS,Black,24,Male,Washington,DC,38.820476,-77.008640,Open/No arrest
52178,Was-001384,20160901,JACKSON,KEVIN,Black,17,Male,Washington,DC,38.866689,-76.982409,Closed by arrest


In [3]:
homicide_df['reported_date'].value_counts()

20171001     71
20170705     38
20151202     34
20140831     34
20160808     33
             ..
20100224      2
20171208      2
20070906      1
201511105     1
201511018     1
Name: reported_date, Length: 4020, dtype: int64

In [4]:
date_list = []
for d in homicide_df['reported_date']:
    date_list.append((str(d)[0:4]))
    
homicide_df['year'] = date_list


year_df = homicide_df

year_df.head()
    


,uid,reported_date,victim_last,victim_first,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition,year
0,Alb-000001,20100504,GARCIA,JUAN,Hispanic,78,Male,Albuquerque,NM,35.095788,-106.538555,Closed without arrest,2010
1,Alb-000002,20100216,MONTOYA,CAMERON,Hispanic,17,Male,Albuquerque,NM,35.056810,-106.715321,Closed by arrest,2010
2,Alb-000003,20100601,SATTERFIELD,VIVIANA,White,15,Female,Albuquerque,NM,35.086092,-106.695568,Closed without arrest,2010
3,Alb-000004,20100101,MENDIOLA,CARLOS,Hispanic,32,Male,Albuquerque,NM,35.078493,-106.556094,Closed by arrest,2010
4,Alb-000005,20100102,MULA,VIVIAN,White,72,Female,Albuquerque,NM,35.130357,-106.580986,Closed without arrest,2010


In [101]:
homicide_df['city'].value_counts()

Chicago           5535
Philadelphia      3037
Houston           2942
Baltimore         2827
Detroit           2519
Los Angeles       2257
St. Louis         1677
Dallas            1567
Memphis           1514
New Orleans       1434
Las Vegas         1381
Washington        1345
Indianapolis      1322
Kansas City       1190
Jacksonville      1168
Milwaukee         1115
Columbus          1084
Atlanta            973
Oakland            947
Phoenix            914
San Antonio        833
Birmingham         800
Nashville          767
Miami              744
Cincinnati         694
Charlotte          687
Oklahoma City      672
San Francisco      663
Pittsburgh         631
New York           627
Boston             614
Tulsa              584
Louisville         576
Fort Worth         549
Buffalo            521
Fresno             487
San Diego          461
Stockton           444
Richmond           429
Baton Rouge        424
Omaha              409
Albuquerque        378
Long Beach         378
Sacramento 

In [5]:
alb = homicide_df.loc[homicide_df["city"] == "Albuquerque", :]
alb.tail()

,uid,reported_date,victim_last,victim_first,victim_race,victim_age,victim_sex,city,state,lat,lon,disposition,year
373,Alb-000374,20171127,RAVENEL,DEZMOND,Unknown,22,Unknown,Albuquerque,NM,35.057319,-106.578808,Closed by arrest,2017
374,Alb-000375,20171128,RAMIREZ,THOMAS,Unknown,Unknown,Unknown,Albuquerque,NM,35.123837,-106.645503,Open/No arrest,2017
375,Alb-000376,20171129,CHAVEZ,MARQUS,Unknown,Unknown,Unknown,Albuquerque,NM,35.083031,-106.558515,Open/No arrest,2017
376,Alb-000377,20171202,MALDONADO,BENJAMIN,Unknown,31,Unknown,Albuquerque,NM,35.107006,-106.513576,Closed by arrest,2017
377,Alb-000378,20171223,WILLIS,AUDRA,Unknown,39,Unknown,Albuquerque,NM,35.058604,-106.490054,Open/No arrest,2017


In [6]:
race_list = []
for u in alb['victim_race'].unique():
    race_list.append(u)
    
race_list

['Hispanic', 'White', 'Other', 'Black', 'Asian', 'Unknown']

In [8]:
city_list = []

for c in homicide_df['city'].unique():
    city_list.append(c)
    

In [20]:
#Main DF
final_list = []

final_df = pd.DataFrame()

for city in range(0, len(city_list)):
    
    #Locates the city data
    loc1 = homicide_df.loc[homicide_df["city"] == city_list[city], :]
    city_dict = {}
    city_size = int(loc1.size/12)
    unsolved_murder_count = 0
    for race in race_list:
        #Locates the race data for each race
        loc2 = loc1.loc[loc1["victim_race"] == race, :]
        total_murders = int(loc2.size/12)
        loc3 = loc2.loc[loc2["disposition"] == 'Closed by arrest']
        solved_murders = int(loc3.size/12)
        unsolved_murders = total_murders - solved_murders
        unsolved_murder_count = unsolved_murder_count + unsolved_murders
        if city_size > 0:
            city_dict[f'% {race}'] = total_murders/city_size
            if unsolved_murders > 0:
                city_dict[f'% Unsolved {race}'] = unsolved_murders/total_murders
            if solved_murders > 0:
                city_dict[f'% Solved {race}'] = solved_murders/total_murders
    df = pd.DataFrame(city_dict, columns = city_dict.keys(), index=[0]) 
    df['City'] = city_list[city]
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    df['Total Murders'] = city_size
    df['Unsolved Murders'] = unsolved_murder_count
    df['Unsolved Murder %'] = unsolved_murder_count/city_size
    final_df = pd.concat([final_df, df])
    final_list.append([city_list[city], city_dict])
        
        
       
final_df

//anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Black,% Unsolved Hispanic,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders
0,0.014670,0.088020,0.178484,0.107579,NaN,0.583333,0.630137,0.704545,0.510870,0.666667,...,0.416667,0.369863,0.295455,0.489130,0.333333,0.381418,Albuquerque,409,0.386308,158
0,0.003795,0.907970,0.019924,0.003795,0.750000,0.617555,0.428571,0.500000,NaN,0.651515,...,0.382445,0.571429,0.500000,NaN,0.348485,0.062619,Atlanta,1054,0.383302,404
0,0.003592,0.918354,0.019922,0.001960,0.636364,0.337482,0.475410,0.333333,NaN,0.570588,...,0.662518,0.524590,0.666667,NaN,0.429412,0.055519,Baltimore,3062,0.645330,1976
0,0.002179,0.884532,0.023965,0.004357,1.000000,0.522167,0.545455,1.000000,NaN,0.648649,...,0.477833,0.454545,NaN,NaN,0.351351,0.080610,Baton Rouge,459,0.461874,212
0,0.000000,0.882217,0.025404,0.005774,NaN,0.571990,0.454545,0.400000,0.666667,0.528571,...,0.428010,0.545455,0.600000,0.333333,0.471429,0.080831,Birmingham,866,0.434180,376
0,0.009023,0.708271,0.183459,0.003008,0.833333,0.416136,0.581967,0.500000,NaN,0.901639,...,0.583864,0.418033,0.500000,1.000000,0.098361,0.091729,Boston,665,0.503759,335
0,0.003546,0.813830,0.069149,0.003546,0.500000,0.346405,0.512821,0.500000,NaN,0.610169,...,0.653595,0.487179,0.500000,1.000000,0.389831,0.104610,Buffalo,564,0.613475,346
0,0.005376,0.706989,0.122312,0.009409,0.250000,0.712928,0.604396,0.571429,NaN,0.792453,...,0.287072,0.395604,0.428571,1.000000,0.207547,0.142473,Charlotte,744,0.299731,223
0,0.004503,0.768012,0.178619,0.001167,0.333333,0.253203,0.268908,0.428571,NaN,0.415162,...,0.746797,0.731092,0.571429,1.000000,0.584838,0.046197,Chicago,5996,0.735991,4413
0,0.001332,0.849534,0.006658,0.010652,NaN,0.523511,0.400000,0.375000,1.000000,0.773196,...,0.476489,0.600000,0.625000,NaN,0.226804,0.129161,Cincinnati,751,0.446072,335


In [21]:
final_df.tail()

,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Black,% Unsolved Hispanic,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders
0,0.003304,0.893722,0.012665,0.002203,0.666667,0.447320,0.478261,0.250000,NaN,0.582278,...,0.552680,0.521739,0.750000,NaN,0.417722,0.087004,St. Louis,1816,0.539648,980
0,0.070686,0.326403,0.461538,0.033264,0.323529,0.420382,0.355856,0.250000,1.0,0.612245,...,0.579618,0.644144,0.750000,NaN,0.387755,0.101871,Stockton,481,0.598753,288
0,0.000000,0.657778,0.097778,0.000000,NaN,0.567568,0.409091,NaN,NaN,0.518519,...,0.432432,0.590909,NaN,NaN,0.481481,0.240000,Tampa,225,0.457778,103
0,0.004747,0.487342,0.071203,0.034810,0.666667,0.626623,0.577778,0.727273,1.0,0.732000,...,0.373377,0.422222,0.272727,NaN,0.268000,0.395570,Tulsa,632,0.329114,208
0,0.010295,0.904598,0.048044,0.004804,0.533333,0.556904,0.571429,0.571429,NaN,0.688889,...,0.443096,0.428571,0.428571,NaN,0.311111,0.030885,Washington,1457,0.437886,638


In [10]:
year_list = []
for year in year_df['year'].unique():
    year_list.append(year)
    
year_list

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2016',
 '2015',
 '2017',
 '2007',
 '2008',
 '2009']

In [80]:
# year_df.loc[homicide_df["year"] == '2010', :]


In [92]:
#Main DF
final_year_list = []

final_year_df = pd.DataFrame()

for i in range(0, len(year_list)):
    

    for city in range(0, len(city_list)):
    
        #Locates the city data
        loc1 = year_df.loc[year_df["city"] == city_list[city], :]
        city_dict = {}
        city_size = int(loc1.size/12)
        unsolved_murder_count = 0
        try: 
            locyear = loc1.loc[loc1['year'] == year_list[i]]
        except:
            locyear = loc1
        for race in race_list:
            #Locates the race data for each race
            loc2 = locyear.loc[locyear["victim_race"] == race, :]
            total_murders = int(loc2.size/12)
            loc3 = loc2.loc[loc2["disposition"] == 'Closed by arrest']
            solved_murders = int(loc3.size/12)
            unsolved_murders = total_murders - solved_murders
            unsolved_murder_count = unsolved_murder_count + unsolved_murders
            if city_size > 0:
                city_dict[f'% {race}'] = total_murders/city_size
                if unsolved_murders > 0:
                    city_dict[f'% Unsolved {race}'] = unsolved_murders/total_murders
                if solved_murders > 0:
                    city_dict[f'% Solved {race}'] = solved_murders/total_murders
        df = pd.DataFrame(city_dict, columns = city_dict.keys(), index=[0]) 
        df['City'] = city_list[city]
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols]
        df['Total Murders'] = city_size
        df['Unsolved Murders'] = unsolved_murder_count
        df['Unsolved Murder %'] = unsolved_murder_count/city_size
        final_year_df = pd.concat([final_year_df, df])
        final_year_list.append([city_list[city], city_dict])
        
        
       
final_df.head()

//anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,City,% Hispanic,% Unsolved Hispanic,% Solved Hispanic,% White,% Unsolved White,% Solved White,% Other,% Unsolved Other,% Solved Other,% Black,% Solved Black,% Asian,% Unsolved Asian,% Unknown,Total Murders,Unsolved Murders,Unsolved Murder %
0,Albuquerque,0.046455,0.263158,0.736842,0.041565,0.529412,0.470588,0.01467,0.166667,0.833333,0.00978,1.0,0.002445,1.0,0.0,409,16,0.03912


In [11]:
final_year_df = pd.DataFrame()

print(len(city_list))
print(len(year_list))
print(race_list)
for i in range(0, len(city_list)):
    for j in range(0, len(year_list)):
        loc1 = year_df.loc[year_df["city"] == city_list[i], :]
        loc2 = loc1.loc[loc1['year'] == year_list[j], :]
        city_year_dict = {}
        year_murder_count = 0
        year_solved_murder_count = 0
        year_unsolved_murder_count = 0
        city_year_size = loc2.size
        
        
        for race in race_list:
            loc3 = loc2.loc[loc2["victim_race"] == race, :]
            total_murders = int(loc3.size/12)
            year_murder_count += total_murders
            loc4 = loc3.loc[loc3['disposition'] == 'Closed by arrest', :]
            solved_murders = int(loc4.size/12)
            year_solved_murder_count += solved_murders
            unsolved_murders = total_murders - solved_murders
            year_unsolved_murder_count += unsolved_murders
            if city_year_size > 0:
                city_year_dict[f'% {race}'] = total_murders/(city_year_size/12)
                if unsolved_murders > 0:
                    city_year_dict[f'% Unsolved {race}'] = unsolved_murders/total_murders
                if solved_murders > 0:
                    city_year_dict[f'% Solved {race}'] = solved_murders/total_murders
                city_year_dict['City'] = city_list[i]
                city_year_dict['Year'] = year_list[j]
                city_year_dict['Total Murders'] = year_murder_count
                city_year_dict['Unsolved Murders'] = year_unsolved_murder_count
                if year_murder_count > 0:
                    city_year_dict['Unsolved Murder %'] = year_unsolved_murder_count/year_murder_count
                else:
                    city_year_dict['Unsolved Murder %'] = 0 
        df = pd.DataFrame(city_year_dict, columns = city_year_dict.keys(), index=[0])
        final_year_df = pd.concat([final_year_df, df])
                
            
final_year_df
        
    
    

50
11
['Hispanic', 'White', 'Other', 'Black', 'Asian', 'Unknown']


//anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Hispanic,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders,Year
0,0.020513,0.082051,0.389744,0.123077,NaN,1.000000,0.736842,0.833333,NaN,0.470588,...,0.263158,0.166667,NaN,0.529412,0.348718,Albuquerque,47.0,0.340426,16.0,2010
0,0.000000,0.145749,0.388664,0.121457,NaN,0.500000,0.562500,1.000000,NaN,0.846154,...,0.437500,NaN,NaN,0.153846,0.315789,Albuquerque,40.0,0.300000,12.0,2011
0,0.000000,0.100334,0.381271,0.160535,NaN,0.400000,0.842105,0.750000,1.0,0.533333,...,0.157895,0.250000,NaN,0.466667,0.301003,Albuquerque,48.0,0.312500,15.0,2012
0,0.000000,0.111888,0.419580,0.083916,NaN,0.750000,0.333333,1.000000,NaN,0.461538,...,0.666667,NaN,NaN,0.538462,0.363636,Albuquerque,35.0,0.514286,18.0,2013
0,0.000000,0.263736,0.000000,0.197802,NaN,0.750000,NaN,0.666667,NaN,0.666667,...,NaN,0.333333,NaN,0.333333,0.494505,Albuquerque,29.0,0.310345,9.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.000000,0.900285,0.051282,0.000000,NaN,0.474684,0.333333,NaN,NaN,1.000000,...,0.666667,NaN,NaN,NaN,0.039886,Washington,174.0,0.511494,89.0,2015
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.010200,0.928177,0.045899,0.000000,0.5,0.571429,0.555556,NaN,NaN,1.000000,...,0.444444,NaN,NaN,NaN,0.010200,Washington,195.0,0.425641,83.0,2007
0,0.004963,0.883375,0.054591,0.009926,NaN,0.589888,0.727273,NaN,NaN,0.714286,...,0.272727,1.000000,NaN,0.285714,0.034739,Washington,199.0,0.407035,81.0,2008


In [12]:
final_year_df.head(20)

,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Hispanic,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders,Year
0,0.020513,0.082051,0.389744,0.123077,NaN,1.000000,0.736842,0.833333,NaN,0.470588,...,0.263158,0.166667,NaN,0.529412,0.348718,Albuquerque,47.0,0.340426,16.0,2010
0,0.000000,0.145749,0.388664,0.121457,NaN,0.500000,0.562500,1.000000,NaN,0.846154,...,0.437500,NaN,NaN,0.153846,0.315789,Albuquerque,40.0,0.300000,12.0,2011
0,0.000000,0.100334,0.381271,0.160535,NaN,0.400000,0.842105,0.750000,1.00,0.533333,...,0.157895,0.250000,NaN,0.466667,0.301003,Albuquerque,48.0,0.312500,15.0,2012
0,0.000000,0.111888,0.419580,0.083916,NaN,0.750000,0.333333,1.000000,NaN,0.461538,...,0.666667,NaN,NaN,0.538462,0.363636,Albuquerque,35.0,0.514286,18.0,2013
0,0.000000,0.263736,0.000000,0.197802,NaN,0.750000,NaN,0.666667,NaN,0.666667,...,NaN,0.333333,NaN,0.333333,0.494505,Albuquerque,29.0,0.310345,9.0,2014
0,0.073260,0.029304,0.000000,0.087912,NaN,0.500000,NaN,0.500000,0.50,0.702128,...,NaN,0.500000,0.50,0.297872,0.688645,Albuquerque,66.0,0.393939,26.0,2016
0,0.000000,0.073846,0.036923,0.110769,NaN,0.250000,NaN,0.500000,0.50,0.687500,...,1.000000,0.500000,0.50,0.312500,0.590769,Albuquerque,52.0,0.423077,22.0,2015
0,0.000000,0.012308,0.012308,0.012308,NaN,NaN,1.000000,NaN,0.52,0.500000,...,NaN,1.000000,0.48,0.500000,0.024615,Albuquerque,80.0,0.487500,39.0,2017
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
latslons = homicide_df.groupby('city').mean()
latslons = latslons.reset_index()
latslons['City'] = latslons['city']
latslons.head()

,city,reported_date,lat,lon,City
0,Albuquerque,2.014018e+07,35.091035,-106.604511,Albuquerque
1,Atlanta,2.011559e+07,33.740595,-84.420151,Atlanta
2,Baltimore,2.012415e+07,39.308105,-76.625437,Baltimore
3,Baton Rouge,2.014150e+07,30.469779,-91.142030,Baton Rouge
4,Birmingham,2.012199e+07,33.518107,-86.825055,Birmingham


In [130]:
# final_year_df = final_df.merge(latslons, on='City')

In [23]:
final_df = final_df.merge(latslons, on='City')


In [24]:
final_df.head()

,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders,city,reported_date,lat,lon
0,0.014670,0.088020,0.178484,0.107579,NaN,0.583333,0.630137,0.704545,0.510870,0.666667,...,0.333333,0.381418,Albuquerque,409,0.386308,158,Albuquerque,2.014018e+07,35.091035,-106.604511
1,0.003795,0.907970,0.019924,0.003795,0.750000,0.617555,0.428571,0.500000,NaN,0.651515,...,0.348485,0.062619,Atlanta,1054,0.383302,404,Atlanta,2.011559e+07,33.740595,-84.420151
2,0.003592,0.918354,0.019922,0.001960,0.636364,0.337482,0.475410,0.333333,NaN,0.570588,...,0.429412,0.055519,Baltimore,3062,0.645330,1976,Baltimore,2.012415e+07,39.308105,-76.625437
3,0.002179,0.884532,0.023965,0.004357,1.000000,0.522167,0.545455,1.000000,NaN,0.648649,...,0.351351,0.080610,Baton Rouge,459,0.461874,212,Baton Rouge,2.014150e+07,30.469779,-91.142030
4,0.000000,0.882217,0.025404,0.005774,NaN,0.571990,0.454545,0.400000,0.666667,0.528571,...,0.471429,0.080831,Birmingham,866,0.434180,376,Birmingham,2.012199e+07,33.518107,-86.825055


In [13]:
final_year_df.head()

,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Hispanic,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders,Year
0,0.020513,0.082051,0.389744,0.123077,NaN,1.00,0.736842,0.833333,NaN,0.470588,...,0.263158,0.166667,NaN,0.529412,0.348718,Albuquerque,47.0,0.340426,16.0,2010
0,0.000000,0.145749,0.388664,0.121457,NaN,0.50,0.562500,1.000000,NaN,0.846154,...,0.437500,NaN,NaN,0.153846,0.315789,Albuquerque,40.0,0.300000,12.0,2011
0,0.000000,0.100334,0.381271,0.160535,NaN,0.40,0.842105,0.750000,1.0,0.533333,...,0.157895,0.250000,NaN,0.466667,0.301003,Albuquerque,48.0,0.312500,15.0,2012
0,0.000000,0.111888,0.419580,0.083916,NaN,0.75,0.333333,1.000000,NaN,0.461538,...,0.666667,NaN,NaN,0.538462,0.363636,Albuquerque,35.0,0.514286,18.0,2013
0,0.000000,0.263736,0.000000,0.197802,NaN,0.75,NaN,0.666667,NaN,0.666667,...,NaN,0.333333,NaN,0.333333,0.494505,Albuquerque,29.0,0.310345,9.0,2014


In [25]:
del final_df['city']
del final_df['reported_date']
final_df.head()

,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders,lat,lon
0,0.014670,0.088020,0.178484,0.107579,NaN,0.583333,0.630137,0.704545,0.510870,0.666667,...,0.295455,0.489130,0.333333,0.381418,Albuquerque,409,0.386308,158,35.091035,-106.604511
1,0.003795,0.907970,0.019924,0.003795,0.750000,0.617555,0.428571,0.500000,NaN,0.651515,...,0.500000,NaN,0.348485,0.062619,Atlanta,1054,0.383302,404,33.740595,-84.420151
2,0.003592,0.918354,0.019922,0.001960,0.636364,0.337482,0.475410,0.333333,NaN,0.570588,...,0.666667,NaN,0.429412,0.055519,Baltimore,3062,0.645330,1976,39.308105,-76.625437
3,0.002179,0.884532,0.023965,0.004357,1.000000,0.522167,0.545455,1.000000,NaN,0.648649,...,NaN,NaN,0.351351,0.080610,Baton Rouge,459,0.461874,212,30.469779,-91.142030
4,0.000000,0.882217,0.025404,0.005774,NaN,0.571990,0.454545,0.400000,0.666667,0.528571,...,0.600000,0.333333,0.471429,0.080831,Birmingham,866,0.434180,376,33.518107,-86.825055


In [136]:
# del final_year_df['city']
# del final_year_df['reported_date']
final_year_df.head(20)

,City,% Hispanic,% Unsolved Hispanic,% Solved Hispanic,% White,% Unsolved White,% Solved White,% Other,% Unsolved Other,% Solved Other,% Black,% Solved Black,% Asian,% Unsolved Asian,% Unknown,Total Murders,Unsolved Murders,Unsolved Murder %,lat,lon
0,Albuquerque,0.046455,0.263158,0.736842,0.041565,0.529412,0.470588,0.01467,0.166667,0.833333,0.00978,1.0,0.002445,1.0,0.0,409,16,0.03912,35.091035,-106.604511


In [140]:
final_

,% Asian,% Black,% Hispanic,% Other,% Solved Asian,% Solved Black,% Solved Hispanic,% Solved Other,% Solved Unknown,% Solved White,...,% Unsolved Black,% Unsolved Hispanic,% Unsolved Other,% Unsolved Unknown,% Unsolved White,% White,City,Total Murders,Unsolved Murder %,Unsolved Murders
0,0.003578,0.893858,0.013119,0.002385,0.666667,0.447632,0.500000,0.250000,NaN,0.582192,...,0.552368,0.500000,0.750000,NaN,0.417808,0.087060,St. Louis,1677,0.539654,905
0,0.072072,0.326577,0.461712,0.033784,0.343750,0.420690,0.356098,0.266667,1.0,0.608696,...,0.579310,0.643902,0.733333,NaN,0.391304,0.103604,Stockton,444,0.599099,266
0,0.000000,0.658654,0.100962,0.000000,NaN,0.569343,0.428571,NaN,NaN,0.520000,...,0.430657,0.571429,NaN,NaN,0.480000,0.240385,Tampa,208,0.456731,95
0,0.005137,0.488014,0.071918,0.035959,0.666667,0.628070,0.571429,0.714286,1.0,0.731602,...,0.371930,0.428571,0.285714,NaN,0.268398,0.395548,Tulsa,584,0.330479,193
0,0.010409,0.904833,0.048327,0.005204,0.571429,0.557108,0.569231,0.571429,NaN,0.690476,...,0.442892,0.430769,0.428571,NaN,0.309524,0.031227,Washington,1345,0.437918,589


In [26]:
#Organizing Our Columns
organized_df = pd.DataFrame()
organized_df['City'] = final_df['City']
organized_df['Total Murders'] = final_df['Total Murders']
organized_df['Unsolved Murders'] = final_df['Unsolved Murders']
organized_df['Unsolved Murder %'] = final_df['Unsolved Murder %']
organized_df['% Total Asian'] = final_df['% Asian']
organized_df['% Unsolved Asian'] = final_df['% Unsolved Asian']
organized_df['% Total Black'] = final_df['% Black']
organized_df['% Unsolved Black'] = final_df['% Unsolved Black']
organized_df['% Total Hispanic'] = final_df['% Hispanic']
organized_df['% Unsolved Hispanic'] = final_df['% Unsolved Hispanic']
organized_df['% Total White'] = final_df['% White']
organized_df['% Unsolved White'] = final_df['% Unsolved White']
organized_df['% Total Other'] = final_df['% Other']
organized_df['% Unsolved Other'] = final_df['% Unsolved Other']
organized_df['% Total Unknown'] = final_df['% Unknown']
organized_df['% Unsolved Unknown'] = final_df['% Unsolved Unknown']


organized_df = organized_df.merge(latslons, on='City')
organized_df.head()

,City,Total Murders,Unsolved Murders,Unsolved Murder %,% Total Asian,% Unsolved Asian,% Total Black,% Unsolved Black,% Total Hispanic,% Unsolved Hispanic,% Total White,% Unsolved White,% Total Other,% Unsolved Other,% Total Unknown,% Unsolved Unknown,city,reported_date,lat,lon
0,Albuquerque,409,158,0.386308,0.014670,1.000000,0.088020,0.416667,0.178484,0.369863,0.381418,0.333333,0.107579,0.295455,0.224939,0.489130,Albuquerque,2.014018e+07,35.091035,-106.604511
1,Atlanta,1054,404,0.383302,0.003795,0.250000,0.907970,0.382445,0.019924,0.571429,0.062619,0.348485,0.003795,0.500000,0.000000,NaN,Atlanta,2.011559e+07,33.740595,-84.420151
2,Baltimore,3062,1976,0.645330,0.003592,0.363636,0.918354,0.662518,0.019922,0.524590,0.055519,0.429412,0.001960,0.666667,0.000000,NaN,Baltimore,2.012415e+07,39.308105,-76.625437
3,Baton Rouge,459,212,0.461874,0.002179,NaN,0.884532,0.477833,0.023965,0.454545,0.080610,0.351351,0.004357,NaN,0.000000,NaN,Baton Rouge,2.014150e+07,30.469779,-91.142030
4,Birmingham,866,376,0.434180,0.000000,NaN,0.882217,0.428010,0.025404,0.545455,0.080831,0.471429,0.005774,0.600000,0.003464,0.333333,Birmingham,2.012199e+07,33.518107,-86.825055


In [18]:
organized_df2 = pd.DataFrame()
organized_df2['City'] = final_year_df['City']
organized_df2['Year'] = final_year_df['Year']
organized_df2['Total Murders'] = final_year_df['Total Murders']
organized_df2['Unsolved Murders'] = final_year_df['Unsolved Murders']
organized_df2['Unsolved Murder %'] = final_year_df['Unsolved Murder %']
organized_df2['% Total Asian'] = final_year_df['% Asian']
organized_df2['% Unsolved Asian'] = final_year_df['% Unsolved Asian']
organized_df2['% Total Black'] = final_year_df['% Black']
organized_df2['% Unsolved Black'] = final_year_df['% Unsolved Black']
organized_df2['% Total Hispanic'] = final_year_df['% Hispanic']
organized_df2['% Unsolved Hispanic'] = final_year_df['% Unsolved Hispanic']
organized_df2['% Total White'] = final_year_df['% White']
organized_df2['% Unsolved White'] = final_year_df['% Unsolved White']
organized_df2['% Total Other'] = final_year_df['% Other']
organized_df2['% Unsolved Other'] = final_year_df['% Unsolved Other']
organized_df2['% Total Unknown'] = final_year_df['% Unknown']
organized_df2['% Unsolved Unknown'] = final_year_df['% Unsolved Unknown']

organized_df2.tail()

,City,Year,Total Murders,Unsolved Murders,Unsolved Murder %,% Total Asian,% Unsolved Asian,% Total Black,% Unsolved Black,% Total Hispanic,% Unsolved Hispanic,% Total White,% Unsolved White,% Total Other,% Unsolved Other,% Total Unknown,% Unsolved Unknown
0,Washington,2015,174.0,89.0,0.511494,0.000000,NaN,0.900285,0.525316,0.051282,0.666667,0.039886,NaN,0.000000,NaN,0.0,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Washington,2007,195.0,83.0,0.425641,0.010200,0.5,0.928177,0.428571,0.045899,0.444444,0.010200,NaN,0.000000,NaN,0.0,NaN
0,Washington,2008,199.0,81.0,0.407035,0.004963,1.0,0.883375,0.410112,0.054591,0.272727,0.034739,0.285714,0.009926,1.0,0.0,NaN
0,Washington,2009,155.0,68.0,0.438710,0.000000,NaN,0.923077,0.437500,0.064103,0.500000,0.000000,NaN,0.006410,NaN,0.0,NaN


In [31]:
organized_df2 = organized_df2.sort_values(by=['City', 'Year'])

organized_df2


,City,Year,Total Murders,Unsolved Murders,Unsolved Murder %,% Total Asian,% Unsolved Asian,% Total Black,% Unsolved Black,% Total Hispanic,% Unsolved Hispanic,% Total White,% Unsolved White,% Total Other,% Unsolved Other,% Total Unknown,% Unsolved Unknown
0,Albuquerque,2010,47.0,16.0,0.340426,0.020513,1.0,0.082051,NaN,0.389744,0.263158,0.348718,0.529412,0.123077,0.166667,0.000000,NaN
0,Albuquerque,2011,40.0,12.0,0.300000,0.000000,NaN,0.145749,0.50,0.388664,0.437500,0.315789,0.153846,0.121457,NaN,0.000000,NaN
0,Albuquerque,2012,48.0,15.0,0.312500,0.000000,NaN,0.100334,0.60,0.381271,0.157895,0.301003,0.466667,0.160535,0.250000,0.020067,NaN
0,Albuquerque,2013,35.0,18.0,0.514286,0.000000,NaN,0.111888,0.25,0.419580,0.666667,0.363636,0.538462,0.083916,NaN,0.000000,NaN
0,Albuquerque,2014,29.0,9.0,0.310345,0.000000,NaN,0.263736,0.25,0.000000,NaN,0.494505,0.333333,0.197802,0.333333,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
del organized_df['city']
del organized_df['reported_date']
organized_df.head()

,City,Total Murders,Unsolved Murders,Unsolved Murder %,% Total Asian,% Unsolved Asian,% Total Black,% Unsolved Black,% Total Hispanic,% Unsolved Hispanic,% Total White,% Unsolved White,% Total Other,% Unsolved Other,% Total Unknown,% Unsolved Unknown,lat,lon
0,Albuquerque,409,158,0.386308,0.014670,1.000000,0.088020,0.416667,0.178484,0.369863,0.381418,0.333333,0.107579,0.295455,0.224939,0.489130,35.091035,-106.604511
1,Atlanta,1054,404,0.383302,0.003795,0.250000,0.907970,0.382445,0.019924,0.571429,0.062619,0.348485,0.003795,0.500000,0.000000,NaN,33.740595,-84.420151
2,Baltimore,3062,1976,0.645330,0.003592,0.363636,0.918354,0.662518,0.019922,0.524590,0.055519,0.429412,0.001960,0.666667,0.000000,NaN,39.308105,-76.625437
3,Baton Rouge,459,212,0.461874,0.002179,NaN,0.884532,0.477833,0.023965,0.454545,0.080610,0.351351,0.004357,NaN,0.000000,NaN,30.469779,-91.142030
4,Birmingham,866,376,0.434180,0.000000,NaN,0.882217,0.428010,0.025404,0.545455,0.080831,0.471429,0.005774,0.600000,0.003464,0.333333,33.518107,-86.825055


In [28]:
organized_df.to_csv('Data_Totals.csv')

In [29]:
organized_df.to_json('Data_Totals.json')

In [32]:
organized_df2.to_csv('Yearly_Totals.csv')


In [43]:
table1 = str(organized_df.to_html())
print(table1.translate({ord('\n'): None}))


<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>City</th>      <th>Total Murders</th>      <th>Unsolved Murders</th>      <th>Unsolved Murder %</th>      <th>% Total Asian</th>      <th>% Unsolved Asian</th>      <th>% Total Black</th>      <th>% Unsolved Black</th>      <th>% Total Hispanic</th>      <th>% Unsolved Hispanic</th>      <th>% Total White</th>      <th>% Unsolved White</th>      <th>% Total Other</th>      <th>% Unsolved Other</th>      <th>% Total Unknown</th>      <th>% Unsolved Unknown</th>      <th>lat</th>      <th>lon</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Albuquerque</td>      <td>409</td>      <td>158</td>      <td>0.386308</td>      <td>0.014670</td>      <td>1.000000</td>      <td>0.088020</td>      <td>0.416667</td>      <td>0.178484</td>      <td>0.369863</td>      <td>0.381418</td>      <td>0.333333</td>      <td>0.107579</td>      <td>0.295455</td>      <td>0.224939</td>     

In [45]:
table2 = str(organized_df2.to_html())
print(table2.translate({ord('\n'): None}))

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>City</th>      <th>Year</th>      <th>Total Murders</th>      <th>Unsolved Murders</th>      <th>Unsolved Murder %</th>      <th>% Total Asian</th>      <th>% Unsolved Asian</th>      <th>% Total Black</th>      <th>% Unsolved Black</th>      <th>% Total Hispanic</th>      <th>% Unsolved Hispanic</th>      <th>% Total White</th>      <th>% Unsolved White</th>      <th>% Total Other</th>      <th>% Unsolved Other</th>      <th>% Total Unknown</th>      <th>% Unsolved Unknown</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Albuquerque</td>      <td>2010</td>      <td>47.0</td>      <td>16.0</td>      <td>0.340426</td>      <td>0.020513</td>      <td>1.000000</td>      <td>0.082051</td>      <td>NaN</td>      <td>0.389744</td>      <td>0.263158</td>      <td>0.348718</td>      <td>0.529412</td>      <td>0.123077</td>      <td>0.166667</td>      <td>0.000000</td>      